# **1. Import libraries and connect to BigQuery**

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [3]:
client = bigquery.Client(project="sample-ecommerce-462804")

# **2. Preview the dataset**

The dataset is obtained from https://developers.google.com/analytics/bigquery/web-ecommerce-demo-dataset.

References for column definitions:
1. [GA4] BigQuery Export schema: https://support.google.com/analytics/answer/7029846
2. [GA4] Recommended events: https://support.google.com/analytics/answer/9267735
3. [GA4] Automatically collected events: https://support.google.com/analytics/answer/9234069


##**2.1. Overview of all columns**

In [3]:
# 1. Preview the dataset
query = """
SELECT *
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131'
LIMIT 5
"""

df = client.query(query).to_dataframe()
df

,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,user_pseudo_id,...,user_ltv,device,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items
0,20201231,1609394908977464,first_visit,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,1187037987,<NA>,None,1005538.3649404164,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
1,20201231,1609394913937736,page_view,"[{'key': 'engagement_time_msec', 'value': {'st...",<NA>,NaN,-6482606431,<NA>,None,1005538.3649404164,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
2,20201231,1609394908977464,session_start,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,1187037987,<NA>,None,1005538.3649404164,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
3,20201231,1609394908977464,page_view,"[{'key': 'page_location', 'value': {'string_va...",<NA>,NaN,1187037987,<NA>,None,1005538.3649404164,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
4,20201231,1609390080342978,first_visit,"[{'key': 'ga_session_number', 'value': {'strin...",<NA>,NaN,-9991068201,<NA>,None,1007822.0143885764,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': '(none)', 'name': '(direct)', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]


In [4]:
# View what columns there are and their data types
df.dtypes

,0
event_date,object
event_timestamp,Int64
event_name,object
event_params,object
event_previous_timestamp,Int64
event_value_in_usd,float64
event_bundle_sequence_id,Int64
event_server_timestamp_offset,Int64
user_id,object
user_pseudo_id,object


##**2.2. The event_name column**

In [5]:
# 2. event_name column
# See what event_names there are and their counts
query = """
SELECT event_name, COUNT(*) AS count
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131'
GROUP BY event_name
ORDER BY count DESC
"""

df = client.query(query).to_dataframe()
df

,event_name,count
0,page_view,1350428
1,user_engagement,1058721
2,scroll,493072
3,view_item,386068
4,session_start,354970
5,first_visit,257462
6,view_promotion,190104
7,add_to_cart,58543
8,begin_checkout,38757
9,select_item,31007


##**2.3. The event_timestamp column**

In [6]:
# 3. event_timestamp column
query = """
SELECT user_pseudo_id, event_name, event_timestamp, DATETIME(TIMESTAMP_MICROS(event_timestamp), 'America/Edmonton') AS event_time
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131'
LIMIT 5
"""

df = client.query(query).to_dataframe()
df

,user_pseudo_id,event_name,event_timestamp,event_time
0,1002973.0469692774,page_view,1609154838103574,2020-12-28 04:27:18.103574
1,1002973.0469692774,first_visit,1609154838103574,2020-12-28 04:27:18.103574
2,1002973.0469692774,session_start,1609154838103574,2020-12-28 04:27:18.103574
3,1003199.3598943909,first_visit,1609137918934244,2020-12-27 23:45:18.934244
4,1003199.3598943909,session_start,1609137918934244,2020-12-27 23:45:18.934244


##**2.4. The event_params column**

In [7]:
# 4. event_params column
# See what event_params there are for page_view
query = """
SELECT user_pseudo_id, event_timestamp, event_name, ep.key, ep.value, ep.value.string_value, ep.value.int_value, ep.value.double_value, ep.value.float_value
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`, UNNEST (event_params) AS ep
WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131' AND event_name = 'page_view'
LIMIT 50
"""

df = client.query(query).to_dataframe()
df

,user_pseudo_id,event_timestamp,event_name,key,value,string_value,int_value,double_value,float_value
0,1013488.1622938896,1608798451074296,page_view,debug_mode,"{'string_value': None, 'int_value': 1, 'float_...",None,1,NaN,NaN
1,1013488.1622938896,1608798451074296,page_view,page_title,"{'string_value': 'Home', 'int_value': None, 'f...",Home,<NA>,NaN,NaN
2,1013488.1622938896,1608798451074296,page_view,session_engaged,"{'string_value': '1', 'int_value': None, 'floa...",1,<NA>,NaN,NaN
3,1013488.1622938896,1608798451074296,page_view,page_location,{'string_value': 'https://shop.googlemerchandi...,https://shop.googlemerchandisestore.com/,<NA>,NaN,NaN
4,1013488.1622938896,1608798451074296,page_view,page_referrer,"{'string_value': None, 'int_value': None, 'flo...",None,<NA>,NaN,NaN
5,1013488.1622938896,1608798451074296,page_view,all_data,"{'string_value': None, 'int_value': None, 'flo...",None,<NA>,NaN,NaN
6,1013488.1622938896,1608798451074296,page_view,ga_session_number,"{'string_value': None, 'int_value': 1, 'float_...",None,1,NaN,NaN
7,1013488.1622938896,1608798451074296,page_view,ga_session_id,"{'string_value': None, 'int_value': 6798788849...",None,6798788849,NaN,NaN
8,1013488.1622938896,1608798451074296,page_view,clean_event,"{'string_value': 'gtm.js', 'int_value': None, ...",gtm.js,<NA>,NaN,NaN
9,1013488.1622938896,1608798451074296,page_view,engaged_session_event,"{'string_value': None, 'int_value': 1, 'float_...",None,1,NaN,NaN


##**2.5. The items column**

In [8]:
# 5. items column
# See what items params there are for add_to_cart
query = """
SELECT user_pseudo_id, event_timestamp, event_name, i.item_name, i.item_id, i.price, i.price_in_usd, i.quantity
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`, UNNEST (items) AS i
WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131' AND event_name = 'add_to_cart'
LIMIT 50
"""
df = client.query(query).to_dataframe()
df

,user_pseudo_id,event_timestamp,event_name,item_name,item_id,price,price_in_usd,quantity
0,3344699.5675826086,1610581636172433,add_to_cart,Google Mens Microfleece Jacket Black,9185141,75.0,NaN,1
1,4553520.4581207598,1610526033871745,add_to_cart,Google Jotter Task Pad,9197992,3.0,NaN,2
2,4553520.4581207598,1610526149380046,add_to_cart,Google Red Speckled Tee,9196578,30.0,NaN,1
3,4553520.4581207598,1610526206313777,add_to_cart,Google Men's Tech Fleece Vest Charcoal,9196592,79.0,NaN,1
4,4644901.1717064819,1610556635277973,add_to_cart,Supernatural Paper Backpack,9197810,58.0,NaN,1
5,7299667.7981365952,1610508840130567,add_to_cart,Google Tee Grey,9188263,22.0,NaN,1
6,8733061.5415155980,1610549090035173,add_to_cart,Google Mesh Bag Red,9188210,6.0,NaN,1
7,15789810.4377143683,1610515013045606,add_to_cart,Supernatural Paper Tote,9196390,28.0,NaN,1
8,27710926.4281558808,1610505849708498,add_to_cart,Google Laptop and Cell Phone Stickers,9180819,3.0,NaN,1
9,46253300.9955720763,1610507490357369,add_to_cart,Google Leather Strap Hat Blue,9195709,17.0,NaN,1


#**3. Construct conversion funnel**

##**3.1 Select a few event_names as our conversion funnel stages**

There are 17 different events. For reference, here are the event_name definitions:
1. **first_visit**: the first time a user visits a website or launches an Android instant app with Analytics enabled
2. **session_start**: when a user engages the app or website. A session ID and session number are generated automatically with each session and associated with each event in the session.
3. **page_view**: each time the page loads or the browser history state is changed by the active site.
4. **user_engagement**: when the app is in the foreground or webpage is in focus for at least one second.
5. **scroll**: the first time a user reaches the bottom of each page (i.e., when a 90% vertical depth becomes visible).
6. **click**: each time a user clicks a link that leads away from the current domain. By default, outbound click events will occur for all links leading away from the current domain. Links to domains configured for cross-domain measurement will not trigger outbound click events.
7. **view_promotion**: views a promotion on your website or app
8. **select_promotion**: selects a promotion
9. **view_search_results**: each time a user performs a site search, indicated by the presence of a URL query parameter
10. **view_item_list**: views a list of items or offerings
11. **select_item**: selects an item from a list of items or offerings
12. **view_item**: views an item
13. **add_to_cart**: adds items to their shopping cart
14. **begin_checkout**: begins checkout
15. **add_shipping_info**: submits their shipping information during checkout
16. **add_payment_info**: submits their payment information during checkout
17. **purchase**: completes a purchase

Based on the event_name definitions, we will proceed with the following event_name sequence as our conversion funnel stages:

session_start → view_item_list → select_item → view_item → add_to_cart → begin_checkout → add_shipping_info → add_payment_info → purchase

##**3.2. Quick check on the selected event_names' distinct user count**

In [9]:
funnel_stages = ["session_start", "view_item_list", "select_item", "view_item", "add_to_cart", "begin_checkout", "add_shipping_info", "add_payment_info", "purchase"]
funnel_stage_counts = []

for stage in funnel_stages:
  query = f"""
    SELECT COUNT(DISTINCT user_pseudo_id) AS user_count
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131' AND event_name = '{stage}'
  """
  df = client.query(query).to_dataframe()
  funnel_stage_counts.append((stage, df['user_count'].iloc[0]))

funnel_stage_counts_df = pd.DataFrame(funnel_stage_counts, columns=['stage', 'user_count'])
funnel_stage_counts_df

,stage,user_count
0,session_start,267116
1,view_item_list,44
2,select_item,13180
3,view_item,61252
4,add_to_cart,12545
5,begin_checkout,9715
6,add_shipping_info,9714
7,add_payment_info,5751
8,purchase,4419


The **view_item_list** event has a relatively low user_count. This may be because it is underreported or simply not a necessary stage for every customer to go through before reaching the view_item stage.

The **select_item** event has a lower user_count than view_item, which may suggest that this is also not a necessary stage for every customer to go through before reaching the subsequent steps.

For simplicity, we will drop the view_item_list and select_item events from our conversion funnel stages. As a result, what we have is

session_start → view_item → add_to_cart → begin_checkout → add_shipping_info → add_payment_info → purchase.

Based on the definitions of these events and the count of distinct user that each event has, we can deduce that most if not all customers must go through these stages following this exact order. Therefore, we will create a closed conversion funnel with these stages.

##**3.3. Create an open funnel for reference**

In [10]:
# For each user, get the first timestamp when they reach each conversion funnel stage
query = """
  CREATE OR REPLACE TABLE sample-ecommerce-462804.ecommerce_dataset.open_funnel_result AS
  WITH funnel_steps AS (
    SELECT user_pseudo_id,
    MIN(CASE WHEN event_name = 'session_start' THEN event_timestamp END) AS session_start_time,
    MIN(CASE WHEN event_name = 'view_item' THEN event_timestamp END) AS view_item_time,
    MIN(CASE WHEN event_name = 'add_to_cart' THEN event_timestamp END) AS add_to_cart_time,
    MIN(CASE WHEN event_name = 'begin_checkout' THEN event_timestamp END) AS begin_checkout_time,
    MIN(CASE WHEN event_name = 'add_shipping_info' THEN event_timestamp END) AS add_shipping_info_time,
    MIN(CASE WHEN event_name = 'add_payment_info' THEN event_timestamp END) AS add_payment_info_time,
    MIN(CASE WHEN event_name = 'purchase' THEN event_timestamp END) AS purchase_time
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131'
    GROUP BY user_pseudo_id
  )

  SELECT * FROM funnel_steps
"""

client.query(query).result()

In [29]:
# Count how many users reached each step
query = """
  SELECT COUNTIF(session_start_time IS NOT NULL) AS session_start_count,
  COUNTIF(view_item_time IS NOT NULL) AS view_item_count,
  COUNTIF(add_to_cart_time IS NOT NULL) AS add_to_cart_count,
  COUNTIF(begin_checkout_time IS NOT NULL) AS begin_checkout_count,
  COUNTIF(add_shipping_info_time IS NOT NULL) AS add_shipping_info_count,
  COUNTIF(add_payment_info_time IS NOT NULL) AS add_payment_info_count,
  COUNTIF(purchase_time IS NOT NULL) AS purchase_count
  FROM `sample-ecommerce-462804.ecommerce_dataset.open_funnel_result`
"""

open_conversion_funnel_df = client.query(query).to_dataframe()
open_conversion_funnel_df

,session_start_count,view_item_count,add_to_cart_count,begin_checkout_count,add_shipping_info_count,add_payment_info_count,purchase_count
0,267116,61252,12545,9715,9714,5751,4419


##**3.4. Create a closed funnel**

In [16]:
query = """
  CREATE OR REPLACE TABLE sample-ecommerce-462804.ecommerce_dataset.closed_funnel_result AS
  WITH funnel_steps AS (
    SELECT user_pseudo_id,
    MIN(CASE WHEN event_name = 'session_start' THEN event_timestamp END) AS session_start_time,
    MIN(CASE WHEN event_name = 'view_item' THEN event_timestamp END) AS view_item_time,
    MIN(CASE WHEN event_name = 'add_to_cart' THEN event_timestamp END) AS add_to_cart_time,
    MIN(CASE WHEN event_name = 'begin_checkout' THEN event_timestamp END) AS begin_checkout_time,
    MIN(CASE WHEN event_name = 'add_shipping_info' THEN event_timestamp END) AS add_shipping_info_time,
    MIN(CASE WHEN event_name = 'add_payment_info' THEN event_timestamp END) AS add_payment_info_time,
    MIN(CASE WHEN event_name = 'purchase' THEN event_timestamp END) AS purchase_time
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131'
    GROUP BY user_pseudo_id
  ),
  closed_funnel AS (
    SELECT *
    FROM funnel_steps
    WHERE session_start_time IS NOT NULL
    AND (view_item_time IS NULL OR view_item_time >= session_start_time)
    AND (add_to_cart_time IS NULL OR add_to_cart_time >= view_item_time)
    AND (begin_checkout_time IS NULL OR begin_checkout_time >= add_to_cart_time)
    AND (add_shipping_info_time IS NULL OR add_shipping_info_time >= begin_checkout_time)
    AND (add_payment_info_time IS NULL OR add_payment_info_time >= add_shipping_info_time)
    AND (purchase_time IS NULL OR purchase_time >= add_payment_info_time)
  )

  SELECT * FROM closed_funnel
"""

client.query(query).result()

In [17]:
# Check if the result is saved properly
query = """
  SELECT *
  FROM `sample-ecommerce-462804.ecommerce_dataset.closed_funnel_result`
  LIMIT 5
"""

df = client.query(query).to_dataframe()
df.head()

,user_pseudo_id,session_start_time,view_item_time,add_to_cart_time,begin_checkout_time,add_shipping_info_time,add_payment_info_time,purchase_time
0,1100441.9272012925,1608889172740439,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2603281.3348169335,1608874176259029,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,4070588.3711566126,1608931142713003,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,5310171.2829669824,1608860304770328,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,6694960.5729059368,1606911337108441,1606911391956827,<NA>,<NA>,<NA>,<NA>,<NA>


In [18]:
# Count how many users reached each step
# All users have a non-null session_start_time so we don't have to check that
query = """
  SELECT COUNT(*) AS session_start_count,
  COUNTIF(view_item_time IS NOT NULL) AS view_item_count,
  COUNTIF(add_to_cart_time IS NOT NULL) AS add_to_cart_count,
  COUNTIF(begin_checkout_time IS NOT NULL) AS begin_checkout_count,
  COUNTIF(add_shipping_info_time IS NOT NULL) AS add_shipping_info_count,
  COUNTIF(add_payment_info_time IS NOT NULL) AS add_payment_info_count,
  COUNTIF(purchase_time IS NOT NULL) AS purchase_count
  FROM `sample-ecommerce-462804.ecommerce_dataset.closed_funnel_result`
"""

conversion_funnel_df = client.query(query).to_dataframe()
conversion_funnel_df

,session_start_count,view_item_count,add_to_cart_count,begin_checkout_count,add_shipping_info_count,add_payment_info_count,purchase_count
0,258227,51539,7834,1090,1090,676,523


In [19]:
# Create a conversion funnel chart
data = dict(
    stage=['session_start', 'view_item', 'add_to_cart', 'begin_checkout', 'add_shipping_info', 'add_payment_info', 'purchase'],
    user_count=[int(conversion_funnel_df['session_start_count'][0]),
                int(conversion_funnel_df['view_item_count'][0]),
                int(conversion_funnel_df['add_to_cart_count'][0]),
                int(conversion_funnel_df['begin_checkout_count'][0]),
                int(conversion_funnel_df['add_shipping_info_count'][0]),
                int(conversion_funnel_df['add_payment_info_count'][0]),
                int(conversion_funnel_df['purchase_count'][0])])

conversion_funnel_percent_df = pd.DataFrame(data)
conversion_funnel_percent_df['conversion_rate'] = conversion_funnel_percent_df['user_count']/conversion_funnel_percent_df['user_count'].shift(1) * 100

fig = go.Figure(go.Funnel(
    y=conversion_funnel_percent_df['stage'],
    x=conversion_funnel_percent_df['user_count'],
    text=[
        f"{convert:.2f}% converted<br>{100-convert:.2f}% dropped"
        if i > 0 else ""
        for i, (count, convert) in enumerate(zip(conversion_funnel_percent_df['user_count'], conversion_funnel_percent_df['conversion_rate'].fillna(0)))
    ]
))

fig.update_layout(title="Conversion Funnel")
fig.update_layout(height=650, width=1200)
fig.show()

In [28]:
# Transpose the closed conversion funnel result to be used in Looker Studio
# Include dates so that we can filter by dates in Looker Studio
# Includes step number so that Looker Studio will display the steps in the correct order
query = """
  CREATE OR REPLACE TABLE sample-ecommerce-462804.ecommerce_dataset.closed_funnel_result_transposed AS
  WITH exploded AS (
    SELECT
      user_pseudo_id,
      DATE(TIMESTAMP_MICROS(session_start_time)) AS session_start_date,
      view_item_time IS NOT NULL AS viewed,
      add_to_cart_time IS NOT NULL AS added,
      begin_checkout_time IS NOT NULL AS checkout_started,
      add_shipping_info_time IS NOT NULL AS shipping_added,
      add_payment_info_time IS NOT NULL AS payment_added,
      purchase_time IS NOT NULL AS purchased
    FROM `sample-ecommerce-462804.ecommerce_dataset.closed_funnel_result`
  )

  SELECT 'session_start' AS event, session_start_date AS event_date, COUNT(*) AS user_count, 1 AS step_order
  FROM exploded
  GROUP BY session_start_date

  UNION ALL

  SELECT 'view_item' AS event, session_start_date AS event_date, COUNT(*) AS user_count, 2 AS step_order
  FROM exploded
  WHERE viewed
  GROUP BY session_start_date

  UNION ALL

  SELECT 'add_to_cart' AS event, session_start_date AS event_date, COUNT(*) AS user_count, 3 AS step_order
  FROM exploded
  WHERE added
  GROUP BY session_start_date

  UNION ALL

  SELECT 'begin_checkout' AS event, session_start_date AS event_date, COUNT(*) AS user_count, 4 AS step_order
  FROM exploded
  WHERE checkout_started
  GROUP BY session_start_date

  UNION ALL

  SELECT 'add_shipping_info' AS event, session_start_date AS event_date, COUNT(*) AS user_count, 5 AS step_order
  FROM exploded
  WHERE shipping_added
  GROUP BY session_start_date

  UNION ALL

  SELECT 'add_payment_info' AS event, session_start_date AS event_date, COUNT(*) AS user_count, 6 AS step_order
  FROM exploded
  WHERE payment_added
  GROUP BY session_start_date

  UNION ALL

  SELECT 'purchase' AS event, session_start_date AS event_date, COUNT(*) AS user_count, 7 AS step_order
  FROM exploded
  WHERE purchased
  GROUP BY session_start_date;
"""

client.query(query).result()

## **3.5. 📑 Overall closed conversion funnel analysis**

Based on the conversion funnel chart above, the biggest drop-off in user count happened at session_start → view_item (80.04% of users dropped off), view_item → add_to_cart (84.80% of users dropped off), and add_to_cart → begin_checkout (86.09% of users dropped off).

**1. 80.04% drop-off from session_start to view_item** \
Possible causes:
1. Low-quality traffic from low-intent sources (e.g. ads that are too broad)
2. Landing pages do not lead to item listing clearly (poor navigation or UX)
3. Users just want to browse around and compare items from different websites

Possible solutions:
1. Improve ad or SEO quality so they target users with a higher interest to purchase Google Merchandise Store's items
2. Improve webpage UI and UX so it's easier for users to arrive at the product detail page
3. Add strong call-to-actions (CTA), discount codes, and highlight best-selling items on the landing page to entice users into exploring more Google Merchandise Store's items

**2. 84.80% drop-off from view_item to add_to_cart** \
Possible causes:
1. Price is too high
2. Insufficient product information
3. Out of stock or low availability
4. Item does not align with user intent

Possible solutions:
1. Add discount codes or offers that are clearly visible on the product detail page
2. Add more high-quality photos, product description, and reviews
3. Ensure that most items are in stock or accurate stock amount is displayed on the product detail page
4. Ensure the ad copy, link captions, or banners leading to the product detail page accurately describe the items users will see

**3. 86.09% drop-off from add_to_cart to begin_checkout** \
Possible causes:
1. Pop-up window asking users to register or log in may cause many users to abandon at this step
2. No urgency (no low stock indicators, limited-time offers)

Possible solutions:
1. Remove the barrier of account creation unless it's necessary
2. Add reviews, return policy, and payment logos on the cart page
3. Add stock counters, time-sensitive deals, or cart expiry notices

**4. 37.98% drop-off from add_shipping_info to add_payment_info** \
It is quite surprising to see that 37.98% of users dropped off from add_shipping_info to add_payment_info. According to GA4 documentation, add_shipping_info is triggered when a user has submitted their shipping information, not when the user reaches the webpage that asks for shipping information. It is interesting to see that almost half of the users dropped off even after they have spent the time writing down their street addresses, postal codes, and so on.

Nonetheless, there are some possible causes for this drop-off:
1. High shipping cost
2. Long delivery time
3. Limited payment options
4. Promo or discount codes fail to work

And the possible solutions are:
1. Offer free shipping on orders that meet a minimum purchase amount
2. Improve logistics operations and ensure the displayed estimated delivery time is realistic and reliable
3. Offer more payment options, such as accepting digital wallets
4. Verify that all discount codes promoted to customers are valid

The exact causes of these relatively large drop-off points can be identified through more in-depth analysis such as deeper data segmentation or user survey, while A/B tests can be used to validate whether specific changes or solutions can address these issues effectively.

#**4. Conversion funnel data segmentation**

##**4.1. Segmentation by device type**

A user may use different device types at different funnel stages. For instance, User A started a session on their phone, viewed an item on desktop, added it to cart on their phone, and later on viewed another item on their phone again.

For easier interpretability, we will assign each user a primary device type, which is the first device type they use. For User A, it will be phone or mobile. We will also implement a closed conversion funnel, which means that only users who followed the specific conversion funnel steps (session_start → view_item → add_to_cart etc.), but are allowed to drop off at any step, are included.

In [30]:
# Check how many users have used different device types throughout the funnel journey
# Only considering the device types associated with the first timestamp of each funnel stage
query = """
  WITH funnel_stage_device AS (
    SELECT
     user_pseudo_id,
     MIN(CASE WHEN event_name = 'session_start' THEN event_timestamp ELSE NULL END) AS session_start_time,
     MIN(CASE WHEN event_name = 'view_item' THEN event_timestamp ELSE NULL END) AS view_item_time,
     MIN(CASE WHEN event_name = 'add_to_cart' THEN event_timestamp ELSE NULL END) AS add_to_cart_time,
     MIN(CASE WHEN event_name = 'begin_checkout' THEN event_timestamp ELSE NULL END) AS begin_checkout_time,
     MIN(CASE WHEN event_name = 'add_shipping_info' THEN event_timestamp ELSE NULL END) AS add_shipping_info_time,
     MIN(CASE WHEN event_name = 'add_payment_info' THEN event_timestamp ELSE NULL END) AS add_payment_info_time,
     MIN(CASE WHEN event_name = 'purchase' THEN event_timestamp ELSE NULL END) AS purchase_time
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131'
    GROUP BY user_pseudo_id
  ),

  first_device_sources AS (
    SELECT
      e.user_pseudo_id,
      COUNT(DISTINCT e.device.category) AS device_count
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` e
    JOIN funnel_stage_device f
      ON e.user_pseudo_id = f.user_pseudo_id
      AND (
      (e.event_name = 'session_start' AND e.event_timestamp = f.session_start_time) OR
      (e.event_name = 'view_item' AND e.event_timestamp = f.view_item_time) OR
      (e.event_name = 'add_to_cart' AND e.event_timestamp = f.add_to_cart_time) OR
      (e.event_name = 'begin_checkout' AND e.event_timestamp = f.begin_checkout_time) OR
      (e.event_name = 'add_shipping_info' AND e.event_timestamp = f.add_shipping_info_time) OR
      (e.event_name = 'add_payment_info' AND e.event_timestamp = f.add_payment_info_time) OR
      (e.event_name = 'purchase' AND e.event_timestamp = f.purchase_time)
      )
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131'
    GROUP BY e.user_pseudo_id
  )

  SELECT device_count, COUNT(*) AS user_count
  FROM first_device_sources
  GROUP BY device_count
  ORDER BY device_count
"""

df = client.query(query).to_dataframe()
df

,device_count,user_count
0,1,267027
1,2,996


In [31]:
query = """
  CREATE OR REPLACE TABLE sample-ecommerce-462804.ecommerce_dataset.closed_funnel_device_result AS
  WITH user_device AS (
    SELECT user_pseudo_id,
    ARRAY_AGG(device.category ORDER BY event_timestamp ASC LIMIT 1)[OFFSET(0)] AS first_device_type
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131' AND event_name = 'session_start'
    GROUP BY user_pseudo_id
  ),

  funnel_steps_device AS (
    SELECT u.user_pseudo_id,
    u.first_device_type AS device_type,
    MIN(CASE WHEN e.event_name = 'session_start' THEN e.event_timestamp END) AS session_start_time,
    MIN(CASE WHEN e.event_name = 'view_item' THEN e.event_timestamp END) AS view_item_time,
    MIN(CASE WHEN e.event_name = 'add_to_cart' THEN e.event_timestamp END) AS add_to_cart_time,
    MIN(CASE WHEN e.event_name = 'begin_checkout' THEN e.event_timestamp END) AS begin_checkout_time,
    MIN(CASE WHEN e.event_name = 'add_shipping_info' THEN e.event_timestamp END) AS add_shipping_info_time,
    MIN(CASE WHEN e.event_name = 'add_payment_info' THEN e.event_timestamp END) AS add_payment_info_time,
    MIN(CASE WHEN e.event_name = 'purchase' THEN e.event_timestamp END) AS purchase_time
    FROM user_device u
    JOIN `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` e
    ON u.user_pseudo_id = e.user_pseudo_id AND u.first_device_type = e.device.category
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131'
    GROUP BY u.user_pseudo_id, device_type
  ),

  closed_funnel_device AS (
    SELECT *
    FROM funnel_steps_device
    WHERE session_start_time IS NOT NULL
    AND (view_item_time IS NULL OR view_item_time >= session_start_time)
    AND (add_to_cart_time IS NULL OR add_to_cart_time >= view_item_time)
    AND (begin_checkout_time IS NULL OR begin_checkout_time >= add_to_cart_time)
    AND (add_shipping_info_time IS NULL OR add_shipping_info_time >= begin_checkout_time)
    AND (add_payment_info_time IS NULL OR add_payment_info_time >= add_shipping_info_time)
    AND (purchase_time IS NULL OR purchase_time >= add_payment_info_time)
  )

  SELECT * FROM closed_funnel_device
"""

client.query(query).result()

In [32]:
# Count how many users reached each step
# All users have a non-null session_start_time so we don't have to check that
query = """
  SELECT device_type,
  COUNT(*) AS session_start_count,
  COUNTIF(view_item_time IS NOT NULL) AS view_item_count,
  COUNTIF(add_to_cart_time IS NOT NULL) AS add_to_cart_count,
  COUNTIF(begin_checkout_time IS NOT NULL) AS begin_checkout_count,
  COUNTIF(add_shipping_info_time IS NOT NULL) AS add_shipping_info_count,
  COUNTIF(add_payment_info_time IS NOT NULL) AS add_payment_info_count,
  COUNTIF(purchase_time IS NOT NULL) AS purchase_count
  FROM `sample-ecommerce-462804.ecommerce_dataset.closed_funnel_device_result`
  GROUP BY device_type
"""

conversion_funnel_device_df = client.query(query).to_dataframe()
conversion_funnel_device_df2 = conversion_funnel_device_df.copy()

conversion_funnel_device_df2['view_item_rate'] = conversion_funnel_device_df2['view_item_count']/conversion_funnel_device_df2['session_start_count'] * 100
conversion_funnel_device_df2['add_to_cart_rate'] = conversion_funnel_device_df2['add_to_cart_count']/conversion_funnel_device_df2['view_item_count'] * 100
conversion_funnel_device_df2['begin_checkout_rate'] = conversion_funnel_device_df2['begin_checkout_count']/conversion_funnel_device_df2['add_to_cart_count'] * 100
conversion_funnel_device_df2['add_shipping_info_rate'] = conversion_funnel_device_df2['add_shipping_info_count']/conversion_funnel_device_df2['begin_checkout_count'] * 100
conversion_funnel_device_df2['add_payment_info_rate'] = conversion_funnel_device_df2['add_payment_info_count']/conversion_funnel_device_df2['add_shipping_info_count'] * 100
conversion_funnel_device_df2['purchase_rate'] = conversion_funnel_device_df2['purchase_count']/conversion_funnel_device_df2['add_payment_info_count'] * 100

new_order = ['device_type', 'session_start_count', 'view_item_count', 'view_item_rate', 'add_to_cart_count', 'add_to_cart_rate', 'begin_checkout_count',
             'begin_checkout_rate', 'add_shipping_info_count', 'add_shipping_info_rate', 'add_payment_info_count', 'add_payment_info_rate',
             'purchase_count', 'purchase_rate']

conversion_funnel_device_df2 = conversion_funnel_device_df2[new_order]
conversion_funnel_device_df2

,device_type,session_start_count,view_item_count,view_item_rate,add_to_cart_count,add_to_cart_rate,begin_checkout_count,begin_checkout_rate,add_shipping_info_count,add_shipping_info_rate,add_payment_info_count,add_payment_info_rate,purchase_count,purchase_rate
0,mobile,102582,20168,19.660369,3029,15.018842,413,13.634863,413,100.0,251,60.774818,192,76.494024
1,desktop,150125,29982,19.971357,4457,14.865586,604,13.551716,604,100.0,369,61.092715,282,76.422764
2,tablet,5820,1157,19.879725,154,13.310285,20,12.987013,20,100.0,11,55.0,8,72.727273


p.s. sum of session_start_count is different from that of closed conversion funnel without any segmentation. Some users' event sequence is messed up if all devices are considered at the same time (in overall closed conversion funnel), such as the user with user_pseudo_id 41179206.8501474).

In [33]:
# Create a conversion funnel chart for each device type
stages = ['session_start', 'view_item', 'add_to_cart', 'begin_checkout', 'add_shipping_info', 'add_payment_info', 'purchase']

for index, row in conversion_funnel_device_df.iterrows():
  user_counts = [int(row['session_start_count']),
                int(row['view_item_count']),
                int(row['add_to_cart_count']),
                int(row['begin_checkout_count']),
                int(row['add_shipping_info_count']),
                int(row['add_payment_info_count']),
                int(row['purchase_count'])]

  data = dict(
    stage = stages,
    user_count= user_counts)

  conversion_funnel_device_percent_df = pd.DataFrame(data)
  conversion_funnel_device_percent_df['conversion_rate'] = conversion_funnel_device_percent_df['user_count']/conversion_funnel_device_percent_df['user_count'].shift(1) * 100

  fig = go.Figure(go.Funnel(
    y=conversion_funnel_device_percent_df['stage'],
    x=conversion_funnel_device_percent_df['user_count'],
    text=[
        f"{convert:.2f}% converted<br>{100-convert:.2f}% dropped"
        if i > 0 else ""
        for i, (count, convert) in enumerate(zip(conversion_funnel_device_percent_df['user_count'], conversion_funnel_device_percent_df['conversion_rate'].fillna(0)))
    ]
  ))

  fig.update_layout(title=f"Conversion Funnel for {row['device_type'].capitalize()}")
  fig.update_layout(height=650, width=1200)
  fig.show()

In [37]:
device_heatmap_df = conversion_funnel_device_df2.set_index('device_type')[['view_item_rate', 'add_to_cart_rate', 'begin_checkout_rate', 'add_shipping_info_rate',
                                                                         'add_payment_info_rate', 'purchase_rate']]

device_heatmap_df.columns = ['View Item', 'Add to Cart', 'Begin Checkout', 'Add Shipping Info', 'Add Payment Info', 'Purchase']
fig = px.imshow(device_heatmap_df,
                labels=dict(x='Funnel Stages', y='Device', color='Conversion Rate'),
                x=device_heatmap_df.columns,
                y=device_heatmap_df.index)

fig.update_layout(title="Conversion Rate Heatmap by Device")
fig.show()

In [39]:
# Create a count and conversion rate table for Looker Studio
query = """
  CREATE OR REPLACE TABLE sample-ecommerce-462804.ecommerce_dataset.closed_funnel_device_funnel_rates AS
  WITH base AS (
    SELECT
      DATE(TIMESTAMP_MICROS(session_start_time)) AS session_start_date,
      device_type,
      COUNT(*) AS session_start_count,
      COUNTIF(view_item_time IS NOT NULL) AS view_item_count,
      COUNTIF(add_to_cart_time IS NOT NULL) AS add_to_cart_count,
      COUNTIF(begin_checkout_time IS NOT NULL) AS begin_checkout_count,
      COUNTIF(add_shipping_info_time IS NOT NULL) AS add_shipping_info_count,
      COUNTIF(add_payment_info_time IS NOT NULL) AS add_payment_info_count,
      COUNTIF(purchase_time IS NOT NULL) AS purchase_count
    FROM `sample-ecommerce-462804.ecommerce_dataset.closed_funnel_device_result`
    WHERE session_start_time IS NOT NULL
    GROUP BY session_start_date, device_type
  )

  SELECT
    session_start_date AS event_date,
    device_type,
    session_start_count,
    view_item_count,
    SAFE_DIVIDE(view_item_count, session_start_count) AS view_item_rate,
    add_to_cart_count,
    SAFE_DIVIDE(add_to_cart_count, view_item_count) AS add_to_cart_rate,
    begin_checkout_count,
    SAFE_DIVIDE(begin_checkout_count, add_to_cart_count) AS begin_checkout_rate,
    add_shipping_info_count,
    SAFE_DIVIDE(add_shipping_info_count, begin_checkout_count) AS add_shipping_info_rate,
    add_payment_info_count,
    SAFE_DIVIDE(add_payment_info_count, add_shipping_info_count) AS add_payment_info_rate,
    purchase_count,
    SAFE_DIVIDE(purchase_count, add_payment_info_count) AS purchase_rate
  FROM base
  ORDER BY session_start_date, session_start_count DESC
  """

client.query(query).result()

## **4.2. 📑 Segmentation by device type analysis**

Compared to desktop and mobile, tablet has more than 3% lower Add Payment Info Rate and Purchase Rate. This suggests that it may worth reviewing the user experience at these stages on tablet and how it differs from that of dektop and mobile. However, it is also important to note that tablet has a much smaller user count, which can result in more exaggerated percentage changes even when the actual difference in user count is small.

Overall, no drastic differences in conversion rates are observed among the three device types. This suggests that the landing page, product detail page, and checkout page and so on offer similar experiences to users regardless of device types.

##**4.3. Segmentation by country**

It is unlikely for users to switch countries mid-session, but we will still check this by running the code below.

In [40]:
# Check how many users have different countries throughout the funnel journey
query = """
  WITH session_users AS (
    SELECT DISTINCT user_pseudo_id
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131' AND event_name = 'session_start'
  ),

  user_country_count AS (
    SELECT e.user_pseudo_id, COUNT(DISTINCT e.geo.country) AS country_count, ARRAY_AGG(DISTINCT e.geo.country) AS countries
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` e
    JOIN session_users s ON e.user_pseudo_id = s.user_pseudo_id
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131'
    GROUP BY e.user_pseudo_id
  )

  SELECT country_count, COUNT(*) AS user_count
  FROM user_country_count
  GROUP BY country_count
  ORDER BY country_count
"""

df = client.query(query).to_dataframe()
df

,country_count,user_count
0,1,267116


In [42]:
query = """
  CREATE OR REPLACE TABLE sample-ecommerce-462804.ecommerce_dataset.closed_funnel_country_result AS
  WITH funnel_steps_country AS (
    SELECT user_pseudo_id,
    geo.country AS country,
    MIN(CASE WHEN event_name = 'session_start' THEN event_timestamp END) AS session_start_time,
    MIN(CASE WHEN event_name = 'view_item' THEN event_timestamp END) AS view_item_time,
    MIN(CASE WHEN event_name = 'add_to_cart' THEN event_timestamp END) AS add_to_cart_time,
    MIN(CASE WHEN event_name = 'begin_checkout' THEN event_timestamp END) AS begin_checkout_time,
    MIN(CASE WHEN event_name = 'add_shipping_info' THEN event_timestamp END) AS add_shipping_info_time,
    MIN(CASE WHEN event_name = 'add_payment_info' THEN event_timestamp END) AS add_payment_info_time,
    MIN(CASE WHEN event_name = 'purchase' THEN event_timestamp END) AS purchase_time
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131'
    GROUP BY user_pseudo_id, country
  ),

  closed_funnel_country AS (
    SELECT *
    FROM funnel_steps_country
    WHERE session_start_time IS NOT NULL
    AND (view_item_time IS NULL OR view_item_time >= session_start_time)
    AND (add_to_cart_time IS NULL OR add_to_cart_time >= view_item_time)
    AND (begin_checkout_time IS NULL OR begin_checkout_time >= add_to_cart_time)
    AND (add_shipping_info_time IS NULL OR add_shipping_info_time >= begin_checkout_time)
    AND (add_payment_info_time IS NULL OR add_payment_info_time >= add_shipping_info_time)
    AND (purchase_time IS NULL OR purchase_time >= add_payment_info_time)
  )

  SELECT * FROM closed_funnel_country
"""

client.query(query).result()

In [43]:
query = """
  SELECT country,
  COUNT(*) AS session_start_count,
  COUNTIF(view_item_time IS NOT NULL) AS view_item_count,
  COUNTIF(add_to_cart_time IS NOT NULL) AS add_to_cart_count,
  COUNTIF(begin_checkout_time IS NOT NULL) AS begin_checkout_count,
  COUNTIF(add_shipping_info_time IS NOT NULL) AS add_shipping_info_count,
  COUNTIF(add_payment_info_time IS NOT NULL) AS add_payment_info_count,
  COUNTIF(purchase_time IS NOT NULL) AS purchase_count
  FROM `sample-ecommerce-462804.ecommerce_dataset.closed_funnel_country_result`
  GROUP BY country
"""

conversion_funnel_country_df = client.query(query).to_dataframe()
conversion_funnel_country_df

,country,session_start_count,view_item_count,add_to_cart_count,begin_checkout_count,add_shipping_info_count,add_payment_info_count,purchase_count
0,(not set),2083,424,62,13,13,7,6
1,Albania,68,13,2,2,2,1,0
2,Algeria,157,39,8,2,2,2,1
3,Argentina,675,130,22,3,3,2,2
4,Armenia,59,13,1,1,1,1,1
...,...,...,...,...,...,...,...,...
104,United Kingdom,8148,1646,251,29,29,20,15
105,United States,113213,22628,3500,504,504,317,249
106,Uruguay,96,13,2,1,1,1,1
107,Venezuela,89,16,1,1,1,0,0


When segmenting by country, the resulting table is sparse. Therefore, we will focus on only the top 5 countries with the largest user counts in our analysis.

In [45]:
conversion_funnel_country_df2 = conversion_funnel_country_df.copy()

conversion_funnel_country_df2['view_item_rate'] = conversion_funnel_country_df2['view_item_count']/conversion_funnel_country_df2['session_start_count'] * 100
conversion_funnel_country_df2['add_to_cart_rate'] = conversion_funnel_country_df2['add_to_cart_count']/conversion_funnel_country_df2['view_item_count'] * 100
conversion_funnel_country_df2['begin_checkout_rate'] = conversion_funnel_country_df2['begin_checkout_count']/conversion_funnel_country_df2['add_to_cart_count'] * 100
conversion_funnel_country_df2['add_shipping_info_rate'] = conversion_funnel_country_df2['add_shipping_info_count']/conversion_funnel_country_df2['begin_checkout_count'] * 100
conversion_funnel_country_df2['add_payment_info_rate'] = conversion_funnel_country_df2['add_payment_info_count']/conversion_funnel_country_df2['add_shipping_info_count'] * 100
conversion_funnel_country_df2['purchase_rate'] = conversion_funnel_country_df2['purchase_count']/conversion_funnel_country_df2['add_payment_info_count'] * 100

new_order = ['country', 'session_start_count', 'view_item_count', 'view_item_rate', 'add_to_cart_count', 'add_to_cart_rate', 'begin_checkout_count',
             'begin_checkout_rate', 'add_shipping_info_count', 'add_shipping_info_rate', 'add_payment_info_count', 'add_payment_info_rate',
             'purchase_count', 'purchase_rate']

conversion_funnel_country_df2 = conversion_funnel_country_df2[new_order]
conversion_funnel_country_df2.sort_values(by='session_start_count', ascending=False, inplace=True)
conversion_funnel_country_df2.head(5)

,country,session_start_count,view_item_count,view_item_rate,add_to_cart_count,add_to_cart_rate,begin_checkout_count,begin_checkout_rate,add_shipping_info_count,add_shipping_info_rate,add_payment_info_count,add_payment_info_rate,purchase_count,purchase_rate
105,United States,113213,22628,19.987104,3500,15.467562,504,14.4,504,100.0,317,62.896825,249,78.548896
43,India,24263,4913,20.248939,752,15.30633,106,14.095745,106,100.0,66,62.264151,49,74.242424
18,Canada,19333,3886,20.100347,588,15.13124,71,12.07483,71,100.0,48,67.605634,34,70.833333
104,United Kingdom,8148,1646,20.201276,251,15.249089,29,11.553785,29,100.0,20,68.965517,15,75.0
33,France,5147,1017,19.759083,148,14.552606,20,13.513514,20,100.0,13,65.0,11,84.615385


In [46]:
# Take only the first five rows
conversion_funnel_country_df3 = conversion_funnel_country_df2.head(5)

# Create a conversion funnel chart for each country
stages = ['session_start', 'view_item', 'add_to_cart', 'begin_checkout', 'add_shipping_info', 'add_payment_info', 'purchase']

for index, row in conversion_funnel_country_df3.iterrows():
  user_counts = [int(row['session_start_count']),
                int(row['view_item_count']),
                int(row['add_to_cart_count']),
                int(row['begin_checkout_count']),
                int(row['add_shipping_info_count']),
                int(row['add_payment_info_count']),
                int(row['purchase_count'])]

  data = dict(
    stage = stages,
    user_count= user_counts)

  conversion_funnel_country_percent_df = pd.DataFrame(data)
  conversion_funnel_country_percent_df['conversion_rate'] = conversion_funnel_country_percent_df['user_count']/conversion_funnel_country_percent_df['user_count'].shift(1) * 100

  fig = go.Figure(go.Funnel(
    y=conversion_funnel_country_percent_df['stage'],
    x=conversion_funnel_country_percent_df['user_count'],
    text=[
        f"{convert:.2f}% converted<br>{100-convert:.2f}% dropped"
        if i > 0 else ""
        for i, (count, convert) in enumerate(zip(conversion_funnel_country_percent_df['user_count'], conversion_funnel_country_percent_df['conversion_rate'].fillna(0)))
    ]
  ))

  fig.update_layout(title=f"Conversion Funnel for {row['country'].capitalize()}")
  fig.update_layout(height=650, width=1200)
  fig.show()

In [47]:
country_heatmap_df = conversion_funnel_country_df3.set_index('country')[['view_item_rate', 'add_to_cart_rate', 'begin_checkout_rate', 'add_shipping_info_rate',
                                                                         'add_payment_info_rate', 'purchase_rate']]

country_heatmap_df.columns = ['View Item', 'Add to Cart', 'Begin Checkout', 'Add Shipping Info', 'Add Payment Info', 'Purchase']
fig = px.imshow(country_heatmap_df,
                labels=dict(x='Funnel Stages', y='Country', color='Conversion Rate'),
                x=country_heatmap_df.columns,
                y=country_heatmap_df.index)

fig.update_layout(title="Conversion Rate Heatmap by Country")
fig.show()

In [48]:
# Create a count and conversion rate table for Looker Studio
query = """
  CREATE OR REPLACE TABLE sample-ecommerce-462804.ecommerce_dataset.closed_funnel_country_funnel_rates AS
  WITH base AS (
    SELECT
      DATE(TIMESTAMP_MICROS(session_start_time)) AS session_start_date,
      country,
      COUNT(*) AS session_start_count,
      COUNTIF(view_item_time IS NOT NULL) AS view_item_count,
      COUNTIF(add_to_cart_time IS NOT NULL) AS add_to_cart_count,
      COUNTIF(begin_checkout_time IS NOT NULL) AS begin_checkout_count,
      COUNTIF(add_shipping_info_time IS NOT NULL) AS add_shipping_info_count,
      COUNTIF(add_payment_info_time IS NOT NULL) AS add_payment_info_count,
      COUNTIF(purchase_time IS NOT NULL) AS purchase_count
    FROM `sample-ecommerce-462804.ecommerce_dataset.closed_funnel_country_result`
    WHERE session_start_time IS NOT NULL
    GROUP BY session_start_date, country
  )

  SELECT
    session_start_date AS event_date,
    country,
    session_start_count,
    view_item_count,
    SAFE_DIVIDE(view_item_count, session_start_count) AS view_item_rate,
    add_to_cart_count,
    SAFE_DIVIDE(add_to_cart_count, view_item_count) AS add_to_cart_rate,
    begin_checkout_count,
    SAFE_DIVIDE(begin_checkout_count, add_to_cart_count) AS begin_checkout_rate,
    add_shipping_info_count,
    SAFE_DIVIDE(add_shipping_info_count, begin_checkout_count) AS add_shipping_info_rate,
    add_payment_info_count,
    SAFE_DIVIDE(add_payment_info_count, add_shipping_info_count) AS add_payment_info_rate,
    purchase_count,
    SAFE_DIVIDE(purchase_count, add_payment_info_count) AS purchase_rate
  FROM base
  ORDER BY session_start_date, session_start_count DESC
  """

client.query(query).result()

##**4.4. Segmentation by continent**

We have already checked that no users changed their countries throughout the funnel journey. Therefore, we will skip the check for users who changed continents throughout the funnel journey.

In [49]:
query = """
  CREATE OR REPLACE TABLE sample-ecommerce-462804.ecommerce_dataset.closed_funnel_continent_result AS
  WITH funnel_steps_continent AS (
    SELECT user_pseudo_id,
    geo.continent AS continent,
    MIN(CASE WHEN event_name = 'session_start' THEN event_timestamp END) AS session_start_time,
    MIN(CASE WHEN event_name = 'view_item' THEN event_timestamp END) AS view_item_time,
    MIN(CASE WHEN event_name = 'add_to_cart' THEN event_timestamp END) AS add_to_cart_time,
    MIN(CASE WHEN event_name = 'begin_checkout' THEN event_timestamp END) AS begin_checkout_time,
    MIN(CASE WHEN event_name = 'add_shipping_info' THEN event_timestamp END) AS add_shipping_info_time,
    MIN(CASE WHEN event_name = 'add_payment_info' THEN event_timestamp END) AS add_payment_info_time,
    MIN(CASE WHEN event_name = 'purchase' THEN event_timestamp END) AS purchase_time
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131'
    GROUP BY user_pseudo_id, continent
  ),

  closed_funnel_continent AS (
    SELECT *
    FROM funnel_steps_continent
    WHERE session_start_time IS NOT NULL
    AND (view_item_time IS NULL OR view_item_time >= session_start_time)
    AND (add_to_cart_time IS NULL OR add_to_cart_time >= view_item_time)
    AND (begin_checkout_time IS NULL OR begin_checkout_time >= add_to_cart_time)
    AND (add_shipping_info_time IS NULL OR add_shipping_info_time >= begin_checkout_time)
    AND (add_payment_info_time IS NULL OR add_payment_info_time >= add_shipping_info_time)
    AND (purchase_time IS NULL OR purchase_time >= add_payment_info_time)
  )

  SELECT * FROM closed_funnel_continent
"""

client.query(query).result()

In [50]:
query = """
  SELECT continent,
  COUNT(*) AS session_start_count,
  COUNTIF(view_item_time IS NOT NULL) AS view_item_count,
  COUNTIF(add_to_cart_time IS NOT NULL) AS add_to_cart_count,
  COUNTIF(begin_checkout_time IS NOT NULL) AS begin_checkout_count,
  COUNTIF(add_shipping_info_time IS NOT NULL) AS add_shipping_info_count,
  COUNTIF(add_payment_info_time IS NOT NULL) AS add_payment_info_count,
  COUNTIF(purchase_time IS NOT NULL) AS purchase_count
  FROM `sample-ecommerce-462804.ecommerce_dataset.closed_funnel_continent_result`
  GROUP BY continent
"""

conversion_funnel_continent_df = client.query(query).to_dataframe()
conversion_funnel_continent_df2 = conversion_funnel_continent_df.copy()

conversion_funnel_continent_df2['view_item_rate'] = conversion_funnel_continent_df2['view_item_count']/conversion_funnel_continent_df2['session_start_count'] * 100
conversion_funnel_continent_df2['add_to_cart_rate'] = conversion_funnel_continent_df2['add_to_cart_count']/conversion_funnel_continent_df2['view_item_count'] * 100
conversion_funnel_continent_df2['begin_checkout_rate'] = conversion_funnel_continent_df2['begin_checkout_count']/conversion_funnel_continent_df2['add_to_cart_count'] * 100
conversion_funnel_continent_df2['add_shipping_info_rate'] = conversion_funnel_continent_df2['add_shipping_info_count']/conversion_funnel_continent_df2['begin_checkout_count'] * 100
conversion_funnel_continent_df2['add_payment_info_rate'] = conversion_funnel_continent_df2['add_payment_info_count']/conversion_funnel_continent_df2['add_shipping_info_count'] * 100
conversion_funnel_continent_df2['purchase_rate'] = conversion_funnel_continent_df2['purchase_count']/conversion_funnel_continent_df2['add_payment_info_count'] * 100

new_order = ['continent', 'session_start_count', 'view_item_count', 'view_item_rate', 'add_to_cart_count', 'add_to_cart_rate', 'begin_checkout_count',
             'begin_checkout_rate', 'add_shipping_info_count', 'add_shipping_info_rate', 'add_payment_info_count', 'add_payment_info_rate',
             'purchase_count', 'purchase_rate']

conversion_funnel_continent_df2 = conversion_funnel_continent_df2[new_order]
conversion_funnel_continent_df2

,continent,session_start_count,view_item_count,view_item_rate,add_to_cart_count,add_to_cart_rate,begin_checkout_count,begin_checkout_rate,add_shipping_info_count,add_shipping_info_rate,add_payment_info_count,add_payment_info_rate,purchase_count,purchase_rate
0,(not set),583,135,23.156089,16,11.851852,3,18.75,3,100.0,3,100.0,2,66.666667
1,Africa,2661,534,20.067644,78,14.606742,10,12.820513,10,100.0,6,60.0,5,83.333333
2,Americas,142539,28468,19.972078,4393,15.431362,621,14.136126,621,100.0,393,63.285024,307,78.117048
3,Asia,61505,12288,19.978864,1850,15.055339,252,13.621622,252,100.0,141,55.952381,110,78.014184
4,Europe,48257,9576,19.843753,1408,14.703425,192,13.636364,192,100.0,127,66.145833,94,74.015748
5,Oceania,2682,538,20.059657,89,16.542751,12,13.483146,12,100.0,6,50.0,5,83.333333


In [51]:
# Create a conversion funnel chart for each continent
stages = ['session_start', 'view_item', 'add_to_cart', 'begin_checkout', 'add_shipping_info', 'add_payment_info', 'purchase']

for index, row in conversion_funnel_continent_df.iterrows():
  user_counts = [int(row['session_start_count']),
                int(row['view_item_count']),
                int(row['add_to_cart_count']),
                int(row['begin_checkout_count']),
                int(row['add_shipping_info_count']),
                int(row['add_payment_info_count']),
                int(row['purchase_count'])]

  data = dict(
    stage = stages,
    user_count= user_counts)

  conversion_funnel_continent_percent_df = pd.DataFrame(data)
  conversion_funnel_continent_percent_df['conversion_rate'] = conversion_funnel_continent_percent_df['user_count']/conversion_funnel_continent_percent_df['user_count'].shift(1) * 100

  fig = go.Figure(go.Funnel(
    y=conversion_funnel_continent_percent_df['stage'],
    x=conversion_funnel_continent_percent_df['user_count'],
    text=[
        f"{convert:.2f}% converted<br>{100-convert:.2f}% dropped"
        if i > 0 else ""
        for i, (count, convert) in enumerate(zip(conversion_funnel_continent_percent_df['user_count'], conversion_funnel_continent_percent_df['conversion_rate'].fillna(0)))
    ]
  ))

  fig.update_layout(title=f"Conversion Funnel for {row['continent'].capitalize()}")
  fig.update_layout(height=650, width=1200)
  fig.show()

In [52]:
continent_heatmap_df = conversion_funnel_continent_df2.set_index('continent')[['view_item_rate', 'add_to_cart_rate', 'begin_checkout_rate', 'add_shipping_info_rate',
                                                                         'add_payment_info_rate', 'purchase_rate']]

continent_heatmap_df.columns = ['View Item', 'Add to Cart', 'Begin Checkout', 'Add Shipping Info', 'Add Payment Info', 'Purchase']
continent_heatmap_df.drop(index='(not set)', inplace=True)
fig = px.imshow(continent_heatmap_df,
                labels=dict(x='Funnel Stages', y='Continent', color='Conversion Rate'),
                x=continent_heatmap_df.columns,
                y=continent_heatmap_df.index)

fig.update_layout(title="Conversion Rate Heatmap by Continent")
fig.show()

In [53]:
# Create a count and conversion rate table for Looker Studio
query = """
  CREATE OR REPLACE TABLE sample-ecommerce-462804.ecommerce_dataset.closed_funnel_continent_funnel_rates AS
  WITH base AS (
    SELECT
      DATE(TIMESTAMP_MICROS(session_start_time)) AS session_start_date,
      continent,
      COUNT(*) AS session_start_count,
      COUNTIF(view_item_time IS NOT NULL) AS view_item_count,
      COUNTIF(add_to_cart_time IS NOT NULL) AS add_to_cart_count,
      COUNTIF(begin_checkout_time IS NOT NULL) AS begin_checkout_count,
      COUNTIF(add_shipping_info_time IS NOT NULL) AS add_shipping_info_count,
      COUNTIF(add_payment_info_time IS NOT NULL) AS add_payment_info_count,
      COUNTIF(purchase_time IS NOT NULL) AS purchase_count
    FROM `sample-ecommerce-462804.ecommerce_dataset.closed_funnel_continent_result`
    WHERE session_start_time IS NOT NULL
    GROUP BY session_start_date, continent
  )

  SELECT
    session_start_date AS event_date,
    continent,
    session_start_count,
    view_item_count,
    SAFE_DIVIDE(view_item_count, session_start_count) AS view_item_rate,
    add_to_cart_count,
    SAFE_DIVIDE(add_to_cart_count, view_item_count) AS add_to_cart_rate,
    begin_checkout_count,
    SAFE_DIVIDE(begin_checkout_count, add_to_cart_count) AS begin_checkout_rate,
    add_shipping_info_count,
    SAFE_DIVIDE(add_shipping_info_count, begin_checkout_count) AS add_shipping_info_rate,
    add_payment_info_count,
    SAFE_DIVIDE(add_payment_info_count, add_shipping_info_count) AS add_payment_info_rate,
    purchase_count,
    SAFE_DIVIDE(purchase_count, add_payment_info_count) AS purchase_rate
  FROM base
  ORDER BY session_start_date, session_start_count DESC
  """

client.query(query).result()

## **4.5. 📑 Segmentation by country and continent analysis**

Similar to device type segmentation, no drastic differences in conversion rates are observed among countries or continents. This suggests that the landing page, product detail page, and checkout page offer similar experiences to users regardless of country.

As before, we are seeing some relatively larger differences in conversion rates for certain countries or continents, but it is likely the result of exaggerated percentage changes due to smaller user counts. For instance, Oceania has an over 5% lower Add Payment Info Rate compared to other continents. However, if only 1 more user converted from Add Shipping Info to Add Payment Info, Oceania will have an over 8% increase in its Add Payment Info Rate.

##**4.6. Segmentation by traffic source**

Based on the output below, quite a lot of users used multiple traffic sources throughout the closed conversion funnel journey.

In [54]:
# Check how many users have different traffic sources throughout the funnel journey
# Only considering the traffic source associated with the first timestamp of each funnel stage
query = """
  WITH funnel_stage_traffic AS (
    SELECT
     user_pseudo_id,
     MIN(CASE WHEN event_name = 'session_start' THEN event_timestamp ELSE NULL END) AS session_start_time,
     MIN(CASE WHEN event_name = 'view_item' THEN event_timestamp ELSE NULL END) AS view_item_time,
     MIN(CASE WHEN event_name = 'add_to_cart' THEN event_timestamp ELSE NULL END) AS add_to_cart_time,
     MIN(CASE WHEN event_name = 'begin_checkout' THEN event_timestamp ELSE NULL END) AS begin_checkout_time,
     MIN(CASE WHEN event_name = 'add_shipping_info' THEN event_timestamp ELSE NULL END) AS add_shipping_info_time,
     MIN(CASE WHEN event_name = 'add_payment_info' THEN event_timestamp ELSE NULL END) AS add_payment_info_time,
     MIN(CASE WHEN event_name = 'purchase' THEN event_timestamp ELSE NULL END) AS purchase_time
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131'
    GROUP BY user_pseudo_id
  ),

  first_traffic_sources AS (
    SELECT
      e.user_pseudo_id,
      COUNT(DISTINCT e.traffic_source.medium) AS traffic_source_count
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` e
    JOIN funnel_stage_traffic f
      ON e.user_pseudo_id = f.user_pseudo_id
      AND (
      (e.event_name = 'session_start' AND e.event_timestamp = f.session_start_time) OR
      (e.event_name = 'view_item' AND e.event_timestamp = f.view_item_time) OR
      (e.event_name = 'add_to_cart' AND e.event_timestamp = f.add_to_cart_time) OR
      (e.event_name = 'begin_checkout' AND e.event_timestamp = f.begin_checkout_time) OR
      (e.event_name = 'add_shipping_info' AND e.event_timestamp = f.add_shipping_info_time) OR
      (e.event_name = 'add_payment_info' AND e.event_timestamp = f.add_payment_info_time) OR
      (e.event_name = 'purchase' AND e.event_timestamp = f.purchase_time)
      )
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131'
    GROUP BY e.user_pseudo_id
  )

  SELECT traffic_source_count, COUNT(*) AS user_count
  FROM first_traffic_sources
  GROUP BY traffic_source_count
  ORDER BY traffic_source_count
"""

df = client.query(query).to_dataframe()
df

,traffic_source_count,user_count
0,1,260601
1,2,6891
2,3,514
3,4,16
4,5,1


We will only segment by the first traffic source used by each user, just like what we did for device segmentation.

In [56]:
query = """
  CREATE OR REPLACE TABLE sample-ecommerce-462804.ecommerce_dataset.closed_funnel_traffic_source_result AS
  WITH user_traffic_source AS (
    SELECT user_pseudo_id,
    ARRAY_AGG(traffic_source.medium ORDER BY event_timestamp ASC LIMIT 1)[OFFSET(0)] AS first_traffic_source
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131' AND event_name = 'session_start'
    GROUP BY user_pseudo_id
  ),

  funnel_steps_traffic_source AS (
    SELECT u.user_pseudo_id,
    u.first_traffic_source AS traffic_source,
    MIN(CASE WHEN e.event_name = 'session_start' THEN e.event_timestamp END) AS session_start_time,
    MIN(CASE WHEN e.event_name = 'view_item' THEN e.event_timestamp END) AS view_item_time,
    MIN(CASE WHEN e.event_name = 'add_to_cart' THEN e.event_timestamp END) AS add_to_cart_time,
    MIN(CASE WHEN e.event_name = 'begin_checkout' THEN e.event_timestamp END) AS begin_checkout_time,
    MIN(CASE WHEN e.event_name = 'add_shipping_info' THEN e.event_timestamp END) AS add_shipping_info_time,
    MIN(CASE WHEN e.event_name = 'add_payment_info' THEN e.event_timestamp END) AS add_payment_info_time,
    MIN(CASE WHEN e.event_name = 'purchase' THEN e.event_timestamp END) AS purchase_time
    FROM user_traffic_source u
    JOIN `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` e
    ON u.user_pseudo_id = e.user_pseudo_id AND u.first_traffic_source = e.traffic_source.medium
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131'
    GROUP BY u.user_pseudo_id, traffic_source
  ),

  closed_funnel_traffic_source AS (
    SELECT *
    FROM funnel_steps_traffic_source
    WHERE session_start_time IS NOT NULL
    AND (view_item_time IS NULL OR view_item_time >= session_start_time)
    AND (add_to_cart_time IS NULL OR add_to_cart_time >= view_item_time)
    AND (begin_checkout_time IS NULL OR begin_checkout_time >= add_to_cart_time)
    AND (add_shipping_info_time IS NULL OR add_shipping_info_time >= begin_checkout_time)
    AND (add_payment_info_time IS NULL OR add_payment_info_time >= add_shipping_info_time)
    AND (purchase_time IS NULL OR purchase_time >= add_payment_info_time)
  )

  SELECT * FROM closed_funnel_traffic_source
"""

client.query(query).result()

In [57]:
# Count how many users reached each step
# All users have a non-null session_start_time so we don't have to check that
query = """
  SELECT traffic_source,
  COUNT(*) AS session_start_count,
  COUNTIF(view_item_time IS NOT NULL) AS view_item_count,
  COUNTIF(add_to_cart_time IS NOT NULL) AS add_to_cart_count,
  COUNTIF(begin_checkout_time IS NOT NULL) AS begin_checkout_count,
  COUNTIF(add_shipping_info_time IS NOT NULL) AS add_shipping_info_count,
  COUNTIF(add_payment_info_time IS NOT NULL) AS add_payment_info_count,
  COUNTIF(purchase_time IS NOT NULL) AS purchase_count
  FROM `sample-ecommerce-462804.ecommerce_dataset.closed_funnel_traffic_source_result`
  GROUP BY traffic_source
"""

conversion_funnel_traffic_source_df = client.query(query).to_dataframe()
conversion_funnel_traffic_source_df2 = conversion_funnel_traffic_source_df.copy()

conversion_funnel_traffic_source_df2['view_item_rate'] = conversion_funnel_traffic_source_df2['view_item_count']/conversion_funnel_traffic_source_df2['session_start_count'] * 100
conversion_funnel_traffic_source_df2['add_to_cart_rate'] = conversion_funnel_traffic_source_df2['add_to_cart_count']/conversion_funnel_traffic_source_df2['view_item_count'] * 100
conversion_funnel_traffic_source_df2['begin_checkout_rate'] = conversion_funnel_traffic_source_df2['begin_checkout_count']/conversion_funnel_traffic_source_df2['add_to_cart_count'] * 100
conversion_funnel_traffic_source_df2['add_shipping_info_rate'] = conversion_funnel_traffic_source_df2['add_shipping_info_count']/conversion_funnel_traffic_source_df2['begin_checkout_count'] * 100
conversion_funnel_traffic_source_df2['add_payment_info_rate'] = conversion_funnel_traffic_source_df2['add_payment_info_count']/conversion_funnel_traffic_source_df2['add_shipping_info_count'] * 100
conversion_funnel_traffic_source_df2['purchase_rate'] = conversion_funnel_traffic_source_df2['purchase_count']/conversion_funnel_traffic_source_df2['add_payment_info_count'] * 100

new_order = ['traffic_source', 'session_start_count', 'view_item_count', 'view_item_rate', 'add_to_cart_count', 'add_to_cart_rate', 'begin_checkout_count',
             'begin_checkout_rate', 'add_shipping_info_count', 'add_shipping_info_rate', 'add_payment_info_count', 'add_payment_info_rate',
             'purchase_count', 'purchase_rate']

conversion_funnel_traffic_source_df2 = conversion_funnel_traffic_source_df2[new_order]
conversion_funnel_traffic_source_df2

,traffic_source,session_start_count,view_item_count,view_item_rate,add_to_cart_count,add_to_cart_rate,begin_checkout_count,begin_checkout_rate,add_shipping_info_count,add_shipping_info_rate,add_payment_info_count,add_payment_info_rate,purchase_count,purchase_rate
0,(data deleted),2035,670,32.923833,106,15.820896,17,16.037736,17,100.0,14,82.352941,11,78.571429
1,(none),61937,11736,18.948286,1706,14.536469,204,11.957796,204,100.0,112,54.901961,70,62.5
2,<Other>,45950,8402,18.285092,1173,13.960962,133,11.338448,133,100.0,63,47.368421,48,76.190476
3,cpc,13949,2532,18.151839,321,12.677725,36,11.214953,36,100.0,22,61.111111,17,77.272727
4,organic,98617,18646,18.907491,2549,13.670492,287,11.259317,287,100.0,157,54.703833,121,77.070064
5,referral,38151,7479,19.60368,1064,14.226501,139,13.06391,139,100.0,79,56.834532,50,63.291139


In [58]:
# Create a conversion funnel chart for each traffic source
stages = ['session_start', 'view_item', 'add_to_cart', 'begin_checkout', 'add_shipping_info', 'add_payment_info', 'purchase']
acceptable_traffic_sources = ['cpc', 'organic', 'referral']

for index, row in conversion_funnel_traffic_source_df.iterrows():
  if row['traffic_source'] not in acceptable_traffic_sources:
    continue
  user_counts = [int(row['session_start_count']),
                int(row['view_item_count']),
                int(row['add_to_cart_count']),
                int(row['begin_checkout_count']),
                int(row['add_shipping_info_count']),
                int(row['add_payment_info_count']),
                int(row['purchase_count'])]

  data = dict(
    stage = stages,
    user_count= user_counts)

  conversion_funnel_traffic_source_percent_df = pd.DataFrame(data)
  conversion_funnel_traffic_source_percent_df['conversion_rate'] = conversion_funnel_traffic_source_percent_df['user_count']/conversion_funnel_traffic_source_percent_df['user_count'].shift(1) * 100

  fig = go.Figure(go.Funnel(
    y=conversion_funnel_traffic_source_percent_df['stage'],
    x=conversion_funnel_traffic_source_percent_df['user_count'],
    text=[
        f"{convert:.2f}% converted<br>{100-convert:.2f}% dropped"
        if i > 0 else ""
        for i, (count, convert) in enumerate(zip(conversion_funnel_traffic_source_percent_df['user_count'], conversion_funnel_traffic_source_percent_df['conversion_rate'].fillna(0)))
    ]
  ))

  fig.update_layout(title=f"Conversion Funnel for {row['traffic_source'].capitalize()}")
  fig.update_layout(height=650, width=1200)
  fig.show()

In [59]:
traffic_source_heatmap_df = conversion_funnel_traffic_source_df2.set_index('traffic_source')[['view_item_rate', 'add_to_cart_rate', 'begin_checkout_rate', 'add_shipping_info_rate',
                                                                         'add_payment_info_rate', 'purchase_rate']]

traffic_source_heatmap_df.columns = ['View Item', 'Add to Cart', 'Begin Checkout', 'Add Shipping Info', 'Add Payment Info', 'Purchase']
traffic_source_heatmap_df.drop(index=['(data deleted)', '(none)', '<Other>'], inplace=True)

fig = px.imshow(traffic_source_heatmap_df,
                labels=dict(x='Funnel Stages', y='Traffic Source', color='Conversion Rate'),
                x=traffic_source_heatmap_df.columns,
                y=traffic_source_heatmap_df.index)

fig.update_layout(title="Conversion Rate Heatmap by Traffic Source")
fig.show()

In [68]:
# Create a count and conversion rate table for Looker Studio
query = """
  CREATE OR REPLACE TABLE sample-ecommerce-462804.ecommerce_dataset.closed_funnel_traffic_source_funnel_rates AS
  WITH base AS (
    SELECT
      DATE(TIMESTAMP_MICROS(session_start_time)) AS session_start_date,
      traffic_source,
      COUNT(*) AS session_start_count,
      COUNTIF(view_item_time IS NOT NULL) AS view_item_count,
      COUNTIF(add_to_cart_time IS NOT NULL) AS add_to_cart_count,
      COUNTIF(begin_checkout_time IS NOT NULL) AS begin_checkout_count,
      COUNTIF(add_shipping_info_time IS NOT NULL) AS add_shipping_info_count,
      COUNTIF(add_payment_info_time IS NOT NULL) AS add_payment_info_count,
      COUNTIF(purchase_time IS NOT NULL) AS purchase_count
    FROM `sample-ecommerce-462804.ecommerce_dataset.closed_funnel_traffic_source_result`
    WHERE session_start_time IS NOT NULL
    GROUP BY session_start_date, traffic_source
  )

  SELECT
    session_start_date AS event_date,
    traffic_source,
    session_start_count,
    view_item_count,
    SAFE_DIVIDE(view_item_count, session_start_count) AS view_item_rate,
    add_to_cart_count,
    SAFE_DIVIDE(add_to_cart_count, view_item_count) AS add_to_cart_rate,
    begin_checkout_count,
    SAFE_DIVIDE(begin_checkout_count, add_to_cart_count) AS begin_checkout_rate,
    add_shipping_info_count,
    SAFE_DIVIDE(add_shipping_info_count, begin_checkout_count) AS add_shipping_info_rate,
    add_payment_info_count,
    SAFE_DIVIDE(add_payment_info_count, add_shipping_info_count) AS add_payment_info_rate,
    purchase_count,
    SAFE_DIVIDE(purchase_count, add_payment_info_count) AS purchase_rate
  FROM base
  ORDER BY session_start_date, session_start_count DESC
  """

client.query(query).result()

##**4.7. 📑 Segmentation by traffic source analysis**

Users that came from referral had an around 14% lower purchase_rate compared to users that came from cpc and organic. It may be worthwhile to investigate which referral sources (e.g., blogs, influencer links etc.) drive higher-intent users and whether the referral codes that users have attempted to use were working as intended.

Users that came from cpc have an over 4% higher add_payment_info_rate compared to users that came from referral and organic. However, the smaller user count for cpc might have exaggerated this difference in conversion rates a little. In fact, if just 1 fewer person that had added shipping info also added payment info, the add_payment_info_rate will decrease by almost 3%.

##**4.8. Segmentation by item category**

In [62]:
# See how many item categories there are
query = """
  SELECT DISTINCT item.item_category
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`, UNNEST(items) AS item
  WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131'
"""

df = client.query(query).to_dataframe()
df

,item_category
0,
1,Clearance
2,Campus Collection
3,Mugs & Tumblers/
4,Kids
...,...
77,Fun
78,Home/Shop by Brand/#IamRemarkable/
79,Hats
80,Black Lives Matter


Since there are 82 original item categories, it will make our analysis easier by grouping these original item categories into broader categories. Here is the method used to group these item categories:

1. If the original item category has words like "Accessories", "Apparel", "Kid", "Men", or "Hats", we group it under "Apparel".

2. If it includes "Collection", we group it under "Collections".

3. If includes "Backpacks", "Bags", "Drinkware", "Eco-Friendly", "Fun", "Lifetyle", "Mugs", or "Water", we group it under "Lifestyle".

4. If it includes "New", we group it under "New".

5. If it includes "Sale" or "Clearance", we group it under "Sale".

6. If it includes "Android", "Google", "YouTube", or "Shop by Brand", we group it under "Shop by Brand".

7. If it includes "Stationery", "Writing", or "Notebooks", we group it under Stationery.

Anything that does not fit into the above is labeled as "Other".

The broader item categories are inspired by the top navigation menu of the Google Merch Shop website (https://shop.merch.google/).

In [60]:
query = """
  CREATE OR REPLACE TABLE sample-ecommerce-462804.ecommerce_dataset.closed_funnel_item_category_result AS
  WITH user_item_category AS (
  SELECT user_pseudo_id, event_name, event_timestamp,
    CASE
      WHEN item.item_category LIKE 'Accessories%' OR item.item_category LIKE '%Apparel%' OR item.item_category LIKE '%Kid%' OR item.item_category LIKE '%Men%' OR item.item_category LIKE '%Hats%' THEN 'Apparel'
      WHEN item.item_category LIKE '%Collection%' THEN 'Collections'
      WHEN item.item_category LIKE '%Backpacks%' OR item.item_category LIKE '%Bags%' OR item.item_category LIKE '%Drinkware%' OR item.item_category LIKE '%Eco-Friendly%' OR item.item_category LIKE '%Fun%' OR item.item_category LIKE '%Lifestyle%' OR item.item_category LIKE '%Mugs%' OR item.item_category LIKE '%Water%' THEN 'Lifestyle'
      WHEN item.item_category LIKE '%New%' THEN 'New'
      WHEN item.item_category LIKE '%Sale%' OR item.item_category LIKE '%Clearance%' THEN 'Sale'
      WHEN item.item_category LIKE '%Android%' OR item.item_category LIKE '%Google%' OR item.item_category LIKE '%YouTube%' OR item.item_category LIKE '%Shop by Brand%' THEN 'Shop by Brand'
      WHEN item.item_category LIKE '%Stationery%' OR item.item_category LIKE '%Writing%' OR item.item_category LIKE '%Notebooks%' THEN 'Stationery'
      ELSE 'Other'
    END AS item_category
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`, UNNEST(items) AS item
  WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131' AND event_name IN ('view_item', 'add_to_cart', 'begin_checkout', 'purchase')
),

  funnel_steps_item_category AS (
    SELECT user_pseudo_id, item_category,
    MIN(CASE WHEN event_name = 'view_item' THEN event_timestamp END) AS view_item_time,
    MIN(CASE WHEN event_name = 'add_to_cart' THEN event_timestamp END) AS add_to_cart_time,
    MIN(CASE WHEN event_name = 'begin_checkout' THEN event_timestamp END) AS begin_checkout_time,
    MIN(CASE WHEN event_name = 'purchase' THEN event_timestamp END) AS purchase_time
    FROM user_item_category
    WHERE item_category IS NOT NULL
    GROUP BY user_pseudo_id, item_category
  ),

  closed_funnel_item_category AS (
    SELECT *
    FROM funnel_steps_item_category
    WHERE view_item_time IS NOT NULL
      AND (add_to_cart_time IS NULL OR add_to_cart_time >= view_item_time)
      AND (begin_checkout_time IS NULL OR begin_checkout_time >= add_to_cart_time)
      AND (purchase_time IS NULL OR purchase_time >= begin_checkout_time)
  )

  SELECT * FROM closed_funnel_item_category
"""

client.query(query).result()

In [61]:
query = """
  SELECT item_category,
  COUNT(*) AS view_item_count,
  COUNTIF(add_to_cart_time IS NOT NULL) AS add_to_cart_count,
  COUNTIF(begin_checkout_time IS NOT NULL) AS begin_checkout_count,
  COUNTIF(purchase_time IS NOT NULL) AS purchase_count
  FROM `sample-ecommerce-462804.ecommerce_dataset.closed_funnel_item_category_result`
  GROUP BY item_category
"""

conversion_funnel_item_category_df = client.query(query).to_dataframe()
conversion_funnel_item_category_df2 = conversion_funnel_item_category_df.copy()

conversion_funnel_item_category_df2['add_to_cart_rate'] = conversion_funnel_item_category_df2['add_to_cart_count']/conversion_funnel_item_category_df2['view_item_count'] * 100
conversion_funnel_item_category_df2['begin_checkout_rate'] = conversion_funnel_item_category_df2['begin_checkout_count']/conversion_funnel_item_category_df2['add_to_cart_count'] * 100
conversion_funnel_item_category_df2['purchase_rate'] = conversion_funnel_item_category_df2['purchase_count']/conversion_funnel_item_category_df2['begin_checkout_count'] * 100

new_order = ['item_category', 'view_item_count', 'add_to_cart_count', 'add_to_cart_rate', 'begin_checkout_count',
             'begin_checkout_rate','purchase_count', 'purchase_rate']

conversion_funnel_item_category_df2 = conversion_funnel_item_category_df2[new_order]
conversion_funnel_item_category_df2

,item_category,view_item_count,add_to_cart_count,add_to_cart_rate,begin_checkout_count,begin_checkout_rate,purchase_count,purchase_rate
0,Apparel,23323,6703,28.739871,1769,26.391168,1175,66.421707
1,Collections,2526,795,31.472684,332,41.761006,234,70.481928
2,Lifestyle,13919,3213,23.083555,888,27.637722,596,67.117117
3,New,4516,932,20.637733,102,10.944206,73,71.568627
4,Other,8400,1993,23.72619,480,24.084295,357,74.375
5,Sale,7765,2438,31.397296,279,11.443806,195,69.892473
6,Shop by Brand,8629,1706,19.770541,177,10.375147,120,67.79661
7,Stationery,5200,1585,30.480769,217,13.690852,141,64.976959


In [63]:
# Create a conversion funnel chart for each item_category
stages = ['view_item', 'add_to_cart', 'begin_checkout', 'purchase']

for index, row in conversion_funnel_item_category_df.iterrows():
  user_counts = [int(row['view_item_count']),
                int(row['add_to_cart_count']),
                int(row['begin_checkout_count']),
                int(row['purchase_count'])]

  data = dict(
    stage = stages,
    user_count= user_counts)

  conversion_funnel_item_category_percent_df = pd.DataFrame(data)
  conversion_funnel_item_category_percent_df['conversion_rate'] = conversion_funnel_item_category_percent_df['user_count']/conversion_funnel_item_category_percent_df['user_count'].shift(1) * 100

  fig = go.Figure(go.Funnel(
    y=conversion_funnel_item_category_percent_df['stage'],
    x=conversion_funnel_item_category_percent_df['user_count'],
    text=[
        f"{convert:.2f}% converted<br>{100-convert:.2f}% dropped"
        if i > 0 else ""
        for i, (count, convert) in enumerate(zip(conversion_funnel_item_category_percent_df['user_count'], conversion_funnel_item_category_percent_df['conversion_rate'].fillna(0)))
    ]
  ))

  fig.update_layout(title=f"Conversion Funnel for {row['item_category'].capitalize()}")
  fig.update_layout(height=650, width=1200)
  fig.show()

In [67]:
item_category_heatmap_df = conversion_funnel_item_category_df2.set_index('item_category')[['add_to_cart_rate', 'begin_checkout_rate', 'purchase_rate']]

item_category_heatmap_df.columns = ['Add to Cart', 'Begin Checkout', 'Purchase']
fig = px.imshow(item_category_heatmap_df,
                labels=dict(x='Funnel Stages', y='Item Category', color='Conversion Rate'),
                x=item_category_heatmap_df.columns,
                y=item_category_heatmap_df.index)

fig.update_layout(title="Conversion Rate Heatmap by Item Category")
fig.update_layout(height=650, width=1500)
fig.show()

In [70]:
# Create a count and conversion rate table for Looker Studio
query = """
  CREATE OR REPLACE TABLE sample-ecommerce-462804.ecommerce_dataset.closed_funnel_item_category_funnel_rates AS
  WITH base AS (
    SELECT
      DATE(TIMESTAMP_MICROS(view_item_time)) AS view_item_date,
      item_category,
      COUNT(*) AS view_item_count,
      COUNTIF(add_to_cart_time IS NOT NULL) AS add_to_cart_count,
      COUNTIF(begin_checkout_time IS NOT NULL) AS begin_checkout_count,
      COUNTIF(purchase_time IS NOT NULL) AS purchase_count
    FROM `sample-ecommerce-462804.ecommerce_dataset.closed_funnel_item_category_result`
    WHERE view_item_time IS NOT NULL
    GROUP BY view_item_date, item_category
  )

  SELECT
    view_item_date AS event_date,
    item_category,
    view_item_count,
    add_to_cart_count,
    SAFE_DIVIDE(add_to_cart_count, view_item_count) AS add_to_cart_rate,
    begin_checkout_count,
    SAFE_DIVIDE(begin_checkout_count, add_to_cart_count) AS begin_checkout_rate,
    purchase_count,
    SAFE_DIVIDE(purchase_count, begin_checkout_count) AS purchase_rate
  FROM base
  ORDER BY view_item_date, view_item_count DESC
  """

client.query(query).result()

##**4.9. 📑 Segmentation by item category analysis**

None of the rows with data in the Item column have Event Name "session_start", "add_shipping_info", and "add_payment_info". Therefore, we will only consider the funnel stages view_item → add_to_cart → begin_checkout → purchase.

Collections has the best conversion rates across all stages. Therefore, the marketing team might want to focus on promoting or expanding this category further.

Apparel and Lifestyle had moderate Add to Cart Rates and Begin Checkout Rates, but their Purchase Rates were among the lowest three. Since Apparel and Lifestyle had the largest user counts, it might be beneficial to prioritize streamlining user experiences for these two item categories. This might involve addressing potential friction points in the purchase process, such as sizing uncertainty in Apparel, enhancing trust signals by clearly specifying the return policy and displaying good reviews at the checkout page.

Shop by Brand and New have the lowest add_to_cart_rate and begin_checkout_rate, suggesting that they have lower initial engagement. Users that viewed these items might just wanted to explore but not necessarily purchase.

##**4.10. Combine results**

In [71]:
# Combine all data segmentation conversion funnel results for Looker Studio
query = """
  CREATE OR REPLACE TABLE sample-ecommerce-462804.ecommerce_dataset.funnel_combined_all_segments AS

  -- Continent
  SELECT
    'Continent' AS segment_type,
    continent AS segment_value,
    event_date,
    session_start_count,
    view_item_count,
    add_to_cart_count,
    begin_checkout_count,
    add_shipping_info_count,
    add_payment_info_count,
    purchase_count,
  FROM sample-ecommerce-462804.ecommerce_dataset.closed_funnel_continent_funnel_rates

  UNION ALL

  -- Country
  SELECT
    'Country' AS segment_type,
    country AS segment_value,
    event_date,
    session_start_count,
    view_item_count,
    add_to_cart_count,
    begin_checkout_count,
    add_shipping_info_count,
    add_payment_info_count,
    purchase_count,
  FROM sample-ecommerce-462804.ecommerce_dataset.closed_funnel_country_funnel_rates

  UNION ALL

  -- Device type
  SELECT
    'Device type' AS segment_type,
    device_type AS segment_value,
    event_date,
    session_start_count,
    view_item_count,
    add_to_cart_count,
    begin_checkout_count,
    add_shipping_info_count,
    add_payment_info_count,
    purchase_count,
  FROM sample-ecommerce-462804.ecommerce_dataset.closed_funnel_device_funnel_rates

  UNION ALL

  -- Item category (some columns missing)
  SELECT
    'Item category' AS segment_type,
    item_category AS segment_value,
    event_date,
    NULL AS session_start_count,               -- missing column
    view_item_count,
    add_to_cart_count,
    begin_checkout_count,
    NULL AS add_shipping_info_count,          -- missing column
    NULL AS add_payment_info_count,           -- missing column
    purchase_count,
  FROM sample-ecommerce-462804.ecommerce_dataset.closed_funnel_item_category_funnel_rates

  UNION ALL

  -- Traffic source
  SELECT
    'Traffic source' AS segment_type,
    traffic_source AS segment_value,
    event_date,
    session_start_count,
    view_item_count,
    add_to_cart_count,
    begin_checkout_count,
    add_shipping_info_count,
    add_payment_info_count,
    purchase_count,
  FROM sample-ecommerce-462804.ecommerce_dataset.closed_funnel_traffic_source_funnel_rates
"""

client.query(query).result()

#**5. User returning behavior and time to purchase**

##**5.1 Returning behavior**

In [4]:
# Visited but never purchased
# Consider the first return only
query = """
  WITH purchasers AS (
    SELECT DISTINCT user_pseudo_id
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131' AND event_name = 'purchase'
  ),

  first_visit AS (
    SELECT user_pseudo_id,
    MIN(PARSE_DATE('%Y%m%d', event_date)) AS first_visit_time
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131' AND event_name = 'session_start'
    AND user_pseudo_id NOT IN (SELECT * FROM purchasers)
    GROUP BY user_pseudo_id
  ),

  future_visits AS (
    SELECT e.user_pseudo_id, PARSE_DATE('%Y%m%d', e.event_date) AS visit_date, f.first_visit_time
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` e
    JOIN first_visit f ON e.user_pseudo_id = f.user_pseudo_id
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131' AND e.event_name = 'session_start'
    AND PARSE_DATE('%Y%m%d', e.event_date) > f.first_visit_time
  ),

  return_day_count AS (
    SELECT user_pseudo_id,
    MIN(DATE_DIFF(visit_date, first_visit_time, DAY)) AS days_to_return
    FROM future_visits
    GROUP BY user_pseudo_id
  )

  SELECT COUNT(*) AS user_count,
  COUNTIF(days_to_return <= 7) AS returned_within_7_days,
  COUNTIF(days_to_return > 7 AND days_to_return <= 14) AS returned_within_8_to_14_days,
  COUNTIF(days_to_return > 14 AND days_to_return <= 30) AS returned_within_15_to_30_days,
  COUNTIF(days_to_return > 30) AS returned_more_than_30_days
  FROM return_day_count
"""

df = client.query(query).to_dataframe()
visited_returned_df2 = df.copy()

visited_returned_df2['returned_within_7_days_rate'] = visited_returned_df2['returned_within_7_days']/visited_returned_df2['user_count'] * 100
visited_returned_df2['returned_within_8_to_14_days_rate'] = visited_returned_df2['returned_within_8_to_14_days']/visited_returned_df2['user_count'] * 100
visited_returned_df2['returned_within_15_to_30_days_rate'] = visited_returned_df2['returned_within_15_to_30_days']/visited_returned_df2['user_count'] * 100
visited_returned_df2['returned_more_than_30_days_rate'] = visited_returned_df2['returned_more_than_30_days']/visited_returned_df2['user_count'] * 100

new_order = ['user_count', 'returned_within_7_days', 'returned_within_7_days_rate', 'returned_within_8_to_14_days', 'returned_within_8_to_14_days_rate', 'returned_within_15_to_30_days',
             'returned_within_15_to_30_days_rate', 'returned_more_than_30_days', 'returned_more_than_30_days_rate']

visited_returned_df2 = visited_returned_df2[new_order]
visited_returned_df2

,user_count,returned_within_7_days,returned_within_7_days_rate,returned_within_8_to_14_days,returned_within_8_to_14_days_rate,returned_within_15_to_30_days,returned_within_15_to_30_days_rate,returned_more_than_30_days,returned_more_than_30_days_rate
0,25879,19573,75.632752,2588,10.000386,2604,10.062213,1114,4.304649


In [5]:
# Users who have purchased
# Consider the first return only
query = """
  WITH first_purchase AS (
    SELECT user_pseudo_id,
    MIN(PARSE_DATE('%Y%m%d', event_date)) AS first_purchase_time
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131' AND event_name = 'purchase'
    GROUP BY user_pseudo_id
  ),

  future_visits AS (
    SELECT e.user_pseudo_id, PARSE_DATE('%Y%m%d', e.event_date) AS visit_date, f.first_purchase_time
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` e
    JOIN first_purchase f ON e.user_pseudo_id = f.user_pseudo_id
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131' AND e.event_name = 'session_start'
    AND PARSE_DATE('%Y%m%d', e.event_date) > f.first_purchase_time
  ),

  return_day_count AS (
    SELECT user_pseudo_id,
    MIN(DATE_DIFF(visit_date, first_purchase_time, DAY)) AS days_to_return
    FROM future_visits
    GROUP BY user_pseudo_id
  )

  SELECT COUNT(*) AS user_count,
  COUNTIF(days_to_return <= 7) AS returned_within_7_days,
  COUNTIF(days_to_return > 7 AND days_to_return <= 14) AS returned_within_8_to_14_days,
  COUNTIF(days_to_return > 14 AND days_to_return <= 30) AS returned_within_15_to_30_days,
  COUNTIF(days_to_return > 30) AS returned_more_than_30_days
  FROM return_day_count
"""

df = client.query(query).to_dataframe()
purchased_returned_df2 = df.copy()

purchased_returned_df2['returned_within_7_days_rate'] = purchased_returned_df2['returned_within_7_days']/purchased_returned_df2['user_count'] * 100
purchased_returned_df2['returned_within_8_to_14_days_rate'] = purchased_returned_df2['returned_within_8_to_14_days']/purchased_returned_df2['user_count'] * 100
purchased_returned_df2['returned_within_15_to_30_days_rate'] = purchased_returned_df2['returned_within_15_to_30_days']/purchased_returned_df2['user_count'] * 100
purchased_returned_df2['returned_more_than_30_days_rate'] = purchased_returned_df2['returned_more_than_30_days']/purchased_returned_df2['user_count'] * 100

new_order = ['user_count', 'returned_within_7_days', 'returned_within_7_days_rate', 'returned_within_8_to_14_days', 'returned_within_8_to_14_days_rate', 'returned_within_15_to_30_days',
             'returned_within_15_to_30_days_rate', 'returned_more_than_30_days', 'returned_more_than_30_days_rate']

purchased_returned_df2 = purchased_returned_df2[new_order]
purchased_returned_df2

,user_count,returned_within_7_days,returned_within_7_days_rate,returned_within_8_to_14_days,returned_within_8_to_14_days_rate,returned_within_15_to_30_days,returned_within_15_to_30_days_rate,returned_more_than_30_days,returned_more_than_30_days_rate
0,1945,1404,72.18509,253,13.007712,209,10.745501,79,4.061697


In [9]:
data = data = {
    'Return Window': ['≤ 7 days', '8–14 days', '15–30 days', '> 30 days'],
    'Purchasers': [
        purchased_returned_df2['returned_within_7_days_rate'].iloc[0],
        purchased_returned_df2['returned_within_8_to_14_days_rate'].iloc[0],
        purchased_returned_df2['returned_within_15_to_30_days_rate'].iloc[0],
        purchased_returned_df2['returned_more_than_30_days_rate'].iloc[0]
    ],
    'Visitors': [
        visited_returned_df2['returned_within_7_days_rate'].iloc[0],
        visited_returned_df2['returned_within_8_to_14_days_rate'].iloc[0],
        visited_returned_df2['returned_within_15_to_30_days_rate'].iloc[0],
        visited_returned_df2['returned_more_than_30_days_rate'].iloc[0]
    ]}

df = pd.DataFrame(data)
df_melted = df.melt(id_vars='Return Window', var_name='User Type', value_name='Return Rate (%)')

# Create the grouped bar chart
fig = px.bar(
    df_melted,
    x='Return Window',
    y='Return Rate (%)',
    color='User Type',
    barmode='group',
    text='Return Rate (%)',
    title='User Return Timing Breakdown by User Type'
)

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    yaxis_title='Return Rate (%)',
    xaxis_title=None,
    plot_bgcolor='white',
    legend_title='User Type'
)

fig.show()

##**5.2. 📑 Returning behavior analysis**

For both visitors and purchasers, most users that returned did so in the ≤7 days window (around 72–76% of returners). This could imply that users tend to come back shortly after their last session, potentially to complete a purchase, check an order, or continue browsing.

While a slightly higher percentage of visitors return within 7 days (75.6% vs. 72.2%), purchasers exhibit stronger return behavior in the 8–30 day window. This may suggest that although visitors are quicker to return, purchasers are more likely to re-engage over time, potentially indicating that they have stronger engagement and deeper interest or intent to buy again.



##**5.3. Average time to purchase**

In [11]:
query = """
  CREATE OR REPLACE TABLE sample-ecommerce-462804.ecommerce_dataset.user_time_to_purchase AS
  WITH purchase_times AS (
    SELECT
      user_pseudo_id,
      MIN(CASE WHEN event_name = 'session_start' THEN event_timestamp END) AS session_start_time,
      MIN(CASE WHEN event_name = 'purchase' THEN event_timestamp END) AS purchase_time
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20210131'
      AND event_name IN ('session_start', 'purchase')
    GROUP BY user_pseudo_id
  ),

  time_to_purchase AS (
    SELECT
      user_pseudo_id,
      DATE(TIMESTAMP_MICROS(purchase_time)) AS event_date,
      (purchase_time - session_start_time) / 86400000000 AS days_to_purchase
    FROM purchase_times
    WHERE purchase_time IS NOT NULL AND session_start_time IS NOT NULL
  )

  SELECT *
  FROM time_to_purchase
"""

client.query(query).result()

In [12]:
query = """
  SELECT AVG(days_to_purchase) AS avg_days_to_purchase
  FROM `sample-ecommerce-462804.ecommerce_dataset.user_time_to_purchase`
"""

df = client.query(query).to_dataframe()
df

,avg_days_to_purchase
0,5.312935


##**5.4. Average time to purchase analysis**

On average, users take about 5.3 days from their first visit to making their first purchase, suggesting that users may perform comparison shopping or visit the Google Merchandise Store multiple times before making a purchase.